In [1]:
import openai, json


client = openai.OpenAI()
messages = []

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to get the weather of.",
                    }
                },
                "required": ["city"],
            }
        }
    }
]

def get_weather(city):
    return "33 degrees celcius"

FUNCTION_MAP = {
    "get_weather": get_weather,
}


In [2]:
def process_ai_response(message):
    if message.tool_calls:
        # Assistant의 tool_calls를 messages에 추가
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })

        # 각 tool_call 처리
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling {function_name} with {arguments}")

            try:
                arguments = json.loads(tool_call.function.arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            if function_to_run:
                result = function_to_run(**arguments)
                print(f"Ran: {function_name} with args {arguments} and got {result}")
            else:
                result = f"Function {function_name} not found"
                print(f"Error: {result}")

            # Tool 실행 결과를 messages에 추가
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result,
            })

        # Tool 실행 후 AI를 다시 호출하여 최종 응답 받기
        call_ai()
    else:
        # 일반 메시지 처리
        messages.append({
            "role": "assistant",
            "content": message.content or "",
        })
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [3]:
# 초기 시스템 메시지 설정
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can check the weather for cities."
    }
]

# 초기 인사 메시지
print("AI Chat Assistant")
print("-" * 50)
print("Commands: 'exit', 'quit', or 'q' to end the conversation")
print("You can ask me about the weather in any city!")
print("-" * 50)

AI Chat Assistant
--------------------------------------------------
Commands: 'exit', 'quit', or 'q' to end the conversation
You can ask me about the weather in any city!
--------------------------------------------------


In [4]:
while True:
    message = input("Send a message to the AI...")
    if message == "exit" or message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user",
            "content": message,
        })
        print(f"User: {message}")
        call_ai()

User: hi
AI: Hello! How can I assist you today?
User: What's the weather of Seoul ?
Calling get_weather with {"city":"Seoul"}
Ran: get_weather with args {'city': 'Seoul'} and got 33 degrees celcius
AI: The current weather in Seoul is 33 degrees Celsius. If you need more information or updates, feel free to ask!


In [5]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can check the weather for cities.'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user', 'content': "What's the weather of Seoul ?"},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_OrvKR9d2BxAtPkGzitrI804B',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_OrvKR9d2BxAtPkGzitrI804B',
  'name': 'get_weather',
  'content': '33 degrees celcius'},
 {'role': 'assistant',
  'content': 'The current weather in Seoul is 33 degrees Celsius. If you need more information or updates, feel free to ask!'}]